In [1]:
import os, json
import numpy as np
import pandas as pd
from dLoader import DataLoader
from dLoader.utils import regroup

# Get Tickers from Database

In [2]:
directory = os.path.join(os.getcwd(), 'Database')
# Search Param
min_dollar = 0
max_dollar = 10
min_length = 250 * 6
today = pd.Timestamp.now()
# Placeholder
tickers = []
files = os.listdir(directory)
if 'keeper.json' in files:
    path = os.path.join(directory, 'keeper.json')
    db = pd.read_json(path, orient='index')
    # Get pricing
    mask = (db['Latest Price'].between(min_dollar, max_dollar)) & (db['Length in Database'] > min_length)
    ndb = db.loc[mask].copy()
    need_update = (today.day - pd.to_datetime(ndb['Last Updated']).dt.day) > 90
    if need_update.sum() > 0:
        jFile = json.load(open(path, 'r'))
        for ticker in ndb.loc[need_update].index.to_list():
            DataLoader(ticker).update_database()
            jFile[ticker]['Last Updated'] = today.strftime('%Y-%m-%d')
        # Write json file
        json.dump(jFile, open(path, 'w'), indent=4)
    tickers = db.loc[mask].index.to_list()
else:
    jFile = {}
    for file in files:
        name, ext = file.split('.')
        if ext == 'csv':
            df = DataLoader(name).data
            latest_price = df['Close'].iloc[-1]
            jFile[name] = {'Latest Price': latest_price,
                           'Length in Database': len(df),
                           'Last Updated': df.index[-1].strftime('%Y-%m-%d')}
            if len(df) > min_length and (min_dollar < latest_price < max_dollar):
                tickers.append(name)
    # Write json file
    with open(os.path.join(directory, 'keeper.json'), 'w') as outfile:
        json.dump(jFile, outfile, indent=4)

print('Total tickers met parameters in Database: {}'.format(len(tickers)))

Total tickers met parameters in Database: 253


In [69]:
def ATR(df, p=10):
    hl = df['High'] - df['Low']
    lc = df['Low'] - df['Close'].shift(1)
    hc = df['High'] - df['Close'].shift(1)
    tr = np.stack([hl, lc.abs(), hc.abs()], axis=1).mean(1)
    tr = pd.Series(tr, index=df.index).ewm(alpha=1/p, adjust=False).mean()
    return tr

def FinalBand(BUB, BLB, CLOSE, PERIOD):
    FUB = np.zeros(len(BUB))
    FLB = np.zeros(len(BLB))
    for i in range(PERIOD, len(CLOSE)):
        # Current Final Upperband = Current Basic Upperband 
        # If ((Current Basic Upperband < Previous Final Upperband) or 
        # (Previous Close > Previous Final Upperband))
        # else Previous Final Upperband
        FUB[i] = BUB[i] if BUB[i] < FUB[i - 1] or CLOSE[i - 1] > FUB[i - 1] else FUB[i - 1]
        # Current Final Lowerband = Current Basic Lowerband
        # If ((Current Basic Lowerband > Previous Final Lowerband) or 
        # (Previous Close < Previous Final Lowerband))
        # else Previous Final Lowerband
        FLB[i] = BLB[i] if BLB[i] > FLB[i - 1] or CLOSE[i - 1] < FLB[i - 1] else FUB[i - 1]
    return FUB, FLB

def supertrend_values(FUB, FLB, CLOSE, PERIOD):
    ST = np.zeros(len(CLOSE))
    for i in range(PERIOD, len(CLOSE)):
        # If ((Previous SuperTrend = Previous Final Upperband) and 
        # (Current Close <= Current Final Upperband))
        # Current SuperTrend = Current Final Upperband
        if ST[i - 1] == FUB[i - 1] and CLOSE[i] <= FUB[i]:
            ST[i] = FUB[i]
        # Else If ((Previous SuperTrend = Previous Final Upperband) and 
        # (Current Close > Current Final Upperband))
        # Current SuperTrend = Current Final Lowerband
        elif ST[i - 1] == FUB[i - 1] and CLOSE[i] > FUB[i]:
                ST[i] = FLB[i]
        # Else If ((Previous SuperTrend = Previous Final Lowerband) and
        # (Current Close >= Current Final Lowerband))
        # Current SuperTrend = Current Final Lowerband
        elif ST[i - 1] == FLB[i - 1] and CLOSE[i] >= FLB[i]:
            ST[i] = FLB[i]
        # Else IF ((Previous SuperTrend = Previous Final Lowerband) and 
        # (Current Clsoe < Current Final Lowerband))
        # Current SuperTrend = Current Final Upperband
        elif ST[i - 1] == FLB[i - 1] and CLOSE[i] < FLB[i]:
            ST[i] = FUB[i]
    return ST

def SuperTrend(df, p=10, M=3):
    # Close Price
    CLOSE = np.array(df['Close'])
    # ATR indicator
    atr = ATR(df, p=p)
    # (High Price + Low Price) / 2
    MHL = df.loc[:, ['High', 'Low']].mean(1)
    # Basic Upperband and Lowerband
    BUB = np.array(MHL + M * atr)
    BLB = np.array(MHL - M * atr)
    # Final Upperband and Lowerband
    FUB, FLB = FinalBand(BUB, BLB, CLOSE, p)
    # Set SuperTrend Values
    ST = supertrend_values(FUB, FLB, CLOSE, p)
    # Mark trend direction
    STX = np.zeros(len(df))
    STX = np.where((ST > 0.), np.where((CLOSE < ST), 'short', 'long'), 'no signal')
    return STX

In [70]:
def RSI(df, p=14, ema=True):
    delta = df['Close'].diff()
    up = delta.clip(lower=0)
    down = -1 * delta.clip(upper=0)
    if ema:
        ema_up = up.ewm(com=p-1, adjust=True, min_periods=p).mean()
        ema_down = down.ewm(com=p-1, adjust=True, min_periods=p).mean()
    else:
        ema_up = up.rolling(window=p, adjust=False).mean()
        ema_down = down.rolling(window=p, adjust=False).mean()
    r = ema_up / ema_down
    return 100 - (100 / (1 + r))

In [201]:
ticker = np.random.choice(tickers)
data = DataLoader(ticker).get_data('2018-01-01', '2018-12-31')

In [202]:
data['Trigger'] = data['Low'] == data['Low'].rolling(6).min()

In [203]:
sell_limit = 0.02
holding = False
buy_at = 0
capital = 1000
shares = 0
trades = 0
for d, row in data.iterrows():
    if row.Trigger and not holding:
        buy_at = row.Low * 1.01
        shares = capital // buy_at
        shares = 1000 if shares > 1000 else shares
        capital -= buy_at * shares
        print('Buy: ${:.2f}, # Shares: {}, Cost: ${:.2f}'.format(
            buy_at, shares, buy_at * shares))
        holding = True
    elif holding:
        g = row.High / buy_at - 1
        s = row.Low / buy_at - 1
        o = row.Open / buy_at - 1
        if o < -sell_limit:
            sell_at = row.Open
            capital += sell_at * shares
            trades += 1
            print('o%Sell: ${:.2f}, Captial: ${:.2f}, # trades: {}'.format(
                sell_at, capital, trades))
            holding = False
        elif g > sell_limit:
            sell_at = buy_at * (1 + sell_limit)
            capital += sell_at * shares
            trades += 1
            print('+%Sell: ${:.2f}, Captial: ${:.2f}, # trades: {}'.format(
                sell_at, capital, trades))
            holding = False
        elif s < -sell_limit:
            sell_at = buy_at * (1 - sell_limit)
            capital += sell_at * shares
            print('-%Sell: ${:.2f}, Capital: ${:.2f}, # trades: {}'.format(
                sell_at, capital, trades))
            holding = False
        
if holding:
    sell_at = data.iloc[-1]['Close']
    capital += sell_at * shares
    print('eSell: ${:.2f}, Capital: ${:.2f}, # trades: {}'.format(
        sell_at, capital, trades))
    holding = False

Buy: $21.11, # Shares: 47.0, Cost: $992.12
+%Sell: $21.53, Captial: $1019.84, # trades: 1
Buy: $20.51, # Shares: 49.0, Cost: $1005.14
o%Sell: $19.83, Captial: $986.37, # trades: 2
Buy: $17.66, # Shares: 55.0, Cost: $971.57
+%Sell: $18.02, Captial: $1005.80, # trades: 3
Buy: $16.99, # Shares: 59.0, Cost: $1002.30
+%Sell: $17.33, Captial: $1025.85, # trades: 4
Buy: $17.63, # Shares: 58.0, Cost: $1022.81
+%Sell: $17.99, Captial: $1046.30, # trades: 5
Buy: $17.55, # Shares: 59.0, Cost: $1035.67
+%Sell: $17.90, Captial: $1067.02, # trades: 6
Buy: $16.92, # Shares: 63.0, Cost: $1065.80
-%Sell: $16.58, Capital: $1045.70, # trades: 6
Buy: $16.31, # Shares: 64.0, Cost: $1043.94
+%Sell: $16.64, Captial: $1066.58, # trades: 7
Buy: $16.46, # Shares: 64.0, Cost: $1053.63
+%Sell: $16.79, Captial: $1087.65, # trades: 8
Buy: $16.97, # Shares: 64.0, Cost: $1085.95
+%Sell: $17.31, Captial: $1109.37, # trades: 9
Buy: $14.73, # Shares: 75.0, Cost: $1104.43
+%Sell: $15.02, Captial: $1131.46, # trades: 10
B